In [ ]:
%matplotlib qt
import torch as pt
import matplotlib.pyplot as plt
import numpy as np
import cmath
import matplotlib.animation as animation
import scipy as sp

theta_0 = 1 # initial angle of the dipole
kappa = 1 # screening constant
q = 1 # Elementary charge
E_ext = 1E-9 # magnitude external electric field
m = 1 # mass of electron in atomic units
h = 10 # Lateral distance dipole to the axis of movement of the dipoles
l = 2 # distance between charges of dipole
omega = np.sqrt(2*q*E_ext/ (m*l))   
omega_R = 2 * np.sqrt(np.exp(-kappa * h) * kappa * l / (m*h**3))
omega_r = np.sqrt(l * kappa / (m * h**2))

def zeta(t, phi=0):
    return 4 * theta_0 / (m * h**3 * (omega_R**2 - omega**2)) * pt.sin(omega*t)

def zeta_dot(t, phi=0):
    return 4 * theta_0 * omega / (m * h**3 * (omega_R**2 - omega**2)) * pt.cos(omega*t)

def xi(R, t, n=0):
    
    # Calculate probability density
    
    # Complex magnitude part
    Herm = pt.special.hermite_polynomial_h((R-zeta(t)) * np.sqrt(m * omega_R), n)
    exp = pt.exp(-m * omega_R * ((R-zeta(t))**2)/2)
    
    # Complex angle part
    integral = 3 * theta_0 / (m * h ** 6 * (omega_R ** 2 - omega ** 2)) * (2 * t  - (((3 * omega**2 + omega_R) * pt.sin(2 * omega * t)) / ((omega_R**2 - omega**2) * omega)))
    complex_exp = pt.exp(1j * (m * zeta_dot(t) * (R - zeta(t)) + integral))
    
    prob_dens = (Herm * exp)**2
    
    # Normalize probability density over the t-axis
    norm = pt.trapz(prob_dens, R, dim=0)
    return pt.div((Herm * exp), norm) * complex_exp

def get_omega_R():
    return 2 * np.sqrt(np.exp(-kappa * h) * kappa * l / (m*h**3))

def get_omega():
    return np.sqrt(2*q*E_ext / (m*l))

atomic_to_s = 2.418884326505e-17

print(f"{(get_omega_R()):.3E}")
print(f"{(get_omega()):.3E}")

def chi_small(r, n=0):
    r_np = r.numpy()
    chi = pt.exp(-pt.abs(r)/n) * sp.special.genlaguerre(n, 1)(2 * np.abs(r_np)/n)
    norm = pt.trapz(chi**2, r, dim=0)
    return chi / norm

def chi_large(r, n=0):
    chi = pt.special.hermite_polynomial_h(r*np.sqrt(m * omega_r / h), n) * pt.exp(- m * omega_r * r**2 / 2)
    norm = pt.trapz(chi**2, r, dim=0)
    return chi / norm


6.027E-04
3.162E-05


In [3]:
R_arr = pt.linspace(-10, 10, 2000)
t_arr = pt.linspace(0, 15, 2000)

R_grid, t_grid = pt.meshgrid(R_arr, t_arr)
xi_grid = xi(R_grid, t_grid)
xi_grid = pt.abs(xi_grid)**2

fig = plt.figure()
ax = plt.axes(projection = "3d")
ax.plot_surface(R_grid.cpu(), t_grid.cpu(), xi_grid.cpu(), cmap='viridis')
ax.set_xlabel(r'R1 [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
ax.set_ylabel(r't [$time$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
ax.set_zlabel(r'$\xi^2$') 



c:\Users\alekh\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([2000])


Text(0.5, 0, '$\\xi^2$')

In [4]:
R_arr = pt.linspace(-12, 12, 1000)
t_arr = pt.linspace(0, 150, 5000)

R_grid, t_grid = pt.meshgrid(R_arr, t_arr)
xi_grid = xi(R_grid, t_grid)
xi_grid = pt.abs(xi_grid)**2

# Make animated 2d plot moving through time
fig, ax = plt.subplots()
line_abs, = ax.plot(R_arr, pt.abs(xi(R_arr, t_arr[0]))**2, label="abs")
line_real, = ax.plot(R_arr, pt.real(xi(R_arr, t_arr[0]))**2, label="real")
line_im, = ax.plot(R_arr, pt.imag(xi(R_arr, t_arr[0]))**2, label="imag")
ax.set_xlabel(r'R [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
ax.set_ylabel(r'$\xi^2$')
ax.set_xlim(-12, 12)
# ax.set_ylim(-1, 1)
def animate(i):
    line_abs.set_ydata(pt.abs(xi(R_arr, t_arr[i]))**2)
    line_real.set_ydata(pt.real(xi(R_arr, t_arr[i]))**2)
    line_im.set_ydata(pt.imag(xi(R_arr, t_arr[i]))**2)

    # # set limits centered around the x-value of the maximum of the absolute value
    
    # # Find index where absolute value is maximal
    # max_idx = pt.argmax(pt.abs(xi(R_arr, t_arr[i]))**2)
    # max_R = R_arr[max_idx]
    # # find index where absolute value is second maximal
    # max_idx_2 = pt.argsort(pt.abs(xi(R_arr, t_arr[i]))**2)[-2]
    # max_R_2 = R_arr[max_idx_2]
    # middle = (max_R + max_R_2) / 2
    # # ax.set_xlim(middle - 3, middle + 3)

    return line_abs, line_real, line_im

plt.legend()
ani = animation.FuncAnimation(fig=fig, func=animate, frames=750, interval=100, blit=True)
plt.show()


torch.Size([5000])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])


In [5]:
# V(R, t)
def floquet_potential(R, t):
    return np.exp(-kappa * h) * (l * kappa * R**2) / h**3 - 2 * R * theta_0 * pt.sin(omega * t) / h**3 

potential_grid = floquet_potential(R_grid, t_grid)
fig = plt.figure()
ax = plt.axes(projection = "3d")
ax.plot_surface(R_grid.cpu(), t_grid.cpu(), potential_grid.cpu(), cmap='viridis')
ax.set_xlabel(r'R1 [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
ax.set_ylabel(r't [$\hbar / \text{E}_\text{h}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
ax.set_zlabel(r'V [$\text{E}_\text{h}$]') # In units of hartree energy, with E_h = e^2 / (4 pi epsilon_0 a_0) = 4.3597e-18 J

torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size

Text(0.5, 0, 'V [$\\text{E}_\\text{h}$]')

In [6]:
def relative_potential(r):
    return l * kappa * r**2 / (4*h**2) + 1/pt.abs(r) * pt.exp(-kappa * pt.abs(r)) 

r_arr = pt.linspace(-4, 4 , 100)
fig = plt.figure()
y = relative_potential(r_arr).cpu()
plt.plot(r_arr.cpu(), y)
print(y)
plt.xlabel(r'r [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
plt.ylabel(r'V [$\text{E}_\text{h}$]') # In units of hartree energy, with E_h = e^2 / (4 pi epsilon_0 a_0) = 4.3597e-18 J
plt.ylim(0.5, 8)


tensor([ 0.0846,  0.0819,  0.0793,  0.0768,  0.0745,  0.0723,  0.0702,  0.0684,
         0.0667,  0.0651,  0.0638,  0.0627,  0.0619,  0.0613,  0.0609,  0.0609,
         0.0613,  0.0620,  0.0632,  0.0649,  0.0671,  0.0699,  0.0735,  0.0778,
         0.0830,  0.0894,  0.0969,  0.1058,  0.1164,  0.1289,  0.1437,  0.1612,
         0.1819,  0.2066,  0.2360,  0.2713,  0.3139,  0.3656,  0.4292,  0.5081,
         0.6075,  0.7349,  0.9019,  1.1273,  1.4436,  1.9123,  2.6651,  4.0447,
         7.3083, 23.7699, 23.7699,  7.3083,  4.0447,  2.6651,  1.9123,  1.4436,
         1.1273,  0.9019,  0.7349,  0.6075,  0.5081,  0.4292,  0.3656,  0.3139,
         0.2713,  0.2360,  0.2066,  0.1819,  0.1612,  0.1437,  0.1289,  0.1164,
         0.1058,  0.0969,  0.0894,  0.0830,  0.0778,  0.0735,  0.0699,  0.0671,
         0.0649,  0.0632,  0.0620,  0.0613,  0.0609,  0.0609,  0.0613,  0.0619,
         0.0627,  0.0638,  0.0651,  0.0667,  0.0684,  0.0702,  0.0723,  0.0745,
         0.0768,  0.0793,  0.0819,  0.08

(0.5, 8.0)

In [24]:
r_arr = pt.linspace(-5, 5 , 10000)

fig = plt.figure()
y0 = chi_small(r_arr, 0).cpu() 
y1 = chi_small(r_arr, 1).cpu() 
y2 = chi_small(r_arr, 2).cpu()
y3 = chi_small(r_arr, 3).cpu()

plt.xlabel(r'r [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
plt.ylabel(r'$\chi(\text{r})^2$') # In units of hartree energ y, with E_h = e^2 / (4 pi epsilon_0 a_0) = 4.3597e-18 J
plt.plot(r_arr.cpu(), y0, label='n=0')
# plt.plot(r_arr.cpu(), y1, label='n=1')
# plt.plot(r_arr.cpu(), y2, label='n=2')
# plt.plot(r_arr.cpu(), y3, label='n=3')
plt.legend()


C:\Users\alekh\AppData\Local\Temp\ipykernel_1480\1154539450.py:57: RuntimeWarning: divide by zero encountered in divide
  chi = pt.exp(-pt.abs(r)/n) * sp.special.genlaguerre(n, 1)(2 * np.abs(r_sp)/n)


In [22]:
r_arr = pt.linspace(-25, 25 , 10000)

fig = plt.figure()
y1 = chi_large(r_arr, 1).cpu() 
y2 = chi_large(r_arr, 2).cpu()
y3 = chi_large(r_arr, 3).cpu()

plt.xlabel(r'r [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
plt.ylabel(r'$\chi(\text{r})^2$') # In units of hartree energ y, with E_h = e^2 / (4 pi epsilon_0 a_0) = 4.3597e-18 J
plt.plot(r_arr.cpu(), y1, label='n=1')
plt.plot(r_arr.cpu(), y2, label='n=2')
plt.plot(r_arr.cpu(), y3, label='n=3')
plt.legend()